# Install and Import Dependencies


In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://download.pytorch.org/whl/cu124


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [21]:
import torch
import requests
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

# Instantiate Model
We use the BERT-based model for multilingual sentiment analysis.

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

# Test Sentiment Encoding and Prediction
Test the model with a sample review.


In [8]:
tokens = tokenizer.encode("I hated this, absolutely the worst", return_tensors='pt')


In [11]:
results = model(tokens)
int(torch.argmax(results.logits)) + 1

1

# Collect Reviews
Scrape Yelp reviews using BeautifulSoup.



In [15]:
r =  requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
result = soup.find_all('p', {'class': regex})

In [19]:
reviews = [review.text for review in result]

# Load Reviews into a DataFrame and Score Sentiments

In [49]:
df = pd.DataFrame(np.array(reviews), columns = ['Reviews'])
df['Score'] = None

,Reviews,Score
0,Seated without a booking on a super busy Satur...,None
1,The food was decent not great.. We had the gu...,None
2,"Food was okay, guacamole was below average. Se...",None
3,The food and service here was really good. It...,None
4,Visiting from Texas and decided to give this r...,None


In [60]:
scores  = []
for i in range(len(df)) :
  tokens = tokenizer.encode(df.iloc[i, -2], return_tensors='pt')
  result = model(tokens)
  scores.append(int(torch.argmax(result.logits)) + 1)

df['Score'] = scores


# Display the Results

In [62]:
df

,Reviews,Score
0,Seated without a booking on a super busy Satur...,5
1,The food was decent not great.. We had the gu...,2
2,"Food was okay, guacamole was below average. Se...",2
3,The food and service here was really good. It...,5
4,Visiting from Texas and decided to give this r...,5
5,Don't come here expecting legit Mexican food b...,3
6,Out of all the restaurants that I tried in Syd...,5
7,"Great atmosphere, attentive service, solid mar...",3
8,We came here on a Thursday night @ 5pm and by ...,4
9,The food is fresh and tasty. The scallop cevi...,4
